## Setup working enviroment 

In [1]:
!pip install kaggle ijson pyTigerGraph

     |████████████████████████████████| 58 kB 2.2 MB/s eta 0:00:011
     |████████████████████████████████| 106 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 15.7 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 9.0 MB/s  eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=aaf3ab191d708a6b4459b4a0d5c91e8951b21750c87883cc05b334442f3f3d17
  Stored in directory: /home/ubuntu/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
  Created wheel for pyTigerGraph: filename=pyTigerGraph-0.0.6.2-py3-none-any.whl size=15344 sha256=e65d2421ab7e87d0bb61fe7f65fbc87ae84eb558b1be24ac8f9d329fd65f1cee
  Stored in directory: /home/ubuntu/.cache/pip/wheels/ef/8c/96/33d5e1491018b2a64bb3280db5836c76ecb6d6c281feefc17e
  Created wheel for python-slugify: filename=python_slugify-4.0.0-py2.py3-none-any.whl size=5486 sha256=ecc6c0719ee03cd17bf2786282acb0e25a796499bf0c07285ad249911b540065
  Stored in dire

In [2]:
server = 'http://localhost'
print(server+':14240')

http://localhost:14240


In [3]:
import pyTigerGraph as tg

conn = tg.TigerGraphConnection(host=server, graphname='yelp')
shell = tg.Gsql(conn , certNeeded=False)

print(shell.gsql('ls', options=[]))

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
Edge Types: 

Graphs: 
Jobs: 


JSON API version: v2
Syntax version: v1




## Load Yelp Dataset from Kaggle

In [4]:
from ipywidgets import FileUpload
upload = FileUpload()
upload

FileUpload(value={}, description='Upload')

In [6]:
!mkdir ~/.kaggle
with open("/home/ubuntu/.kaggle/kaggle.json", "w+b") as i:
    i.write(upload.data[0])
    
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download yelp-dataset/yelp-dataset
!unzip yelp-dataset

100%|█████████████████████████████████████▉| 4.47G/4.48G [01:29<00:00, 79.5MB/s]
100%|██████████████████████████████████████| 4.48G/4.48G [01:29<00:00, 53.6MB/s]
Archive:  yelp-dataset.zip
  inflating: Dataset_Agreement.pdf   
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


In [8]:
ls -l

total 14911448
-rw-rw-r-- 1 ubuntu ubuntu      41776 Mar 26 01:18 Dataset_Agreement.pdf
-rw-rw-r-- 1 ubuntu ubuntu      21260 Jun 29 21:03 setup_yelp_graph.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  152898689 Mar 26 01:18 yelp_academic_dataset_business.json
-rw-rw-r-- 1 ubuntu ubuntu  449663480 Mar 26 01:18 yelp_academic_dataset_checkin.json
-rw-rw-r-- 1 ubuntu ubuntu 6325565224 Mar 26 01:19 yelp_academic_dataset_review.json
-rw-rw-r-- 1 ubuntu ubuntu  263489322 Mar 26 01:31 yelp_academic_dataset_tip.json
-rw-rw-r-- 1 ubuntu ubuntu 3268069927 Mar 26 01:32 yelp_academic_dataset_user.json
-rw-rw-r-- 1 ubuntu ubuntu 4809540040 Jun 29 21:02 yelp-dataset.zip


In [9]:
!rm yelp-dataset.zip

## Define schema helper functions

In [10]:
%%writefile autotigergraph.py
import ijson

def get_first(filename):

    with open(filename, 'r') as f:
        objects = ijson.items(f, '', multiple_values=True, use_float=True)
        first  = objects.__next__()
        fields = list(zip(range(len(first)), 
                        list(first.keys()), 
                        [type(value).__name__ for value in first.values()]))

    return first, fields


def guess_vertex(json_object, vertex_name, guess_fields=None):

    if guess_fields == None:
        guess_fields = json_object.keys()
    
    type_translate = {'str': 'STRING', 'dict': 'STRING', 'float': 'DOUBLE', 
                      'bool': 'BOOL', 'int': 'INT'}

    gsql_cmd  = 'CREATE VERTEX ' + vertex_name + ' (PRIMARY_ID ' 
    
    for field in guess_fields:

        if field == 'date':
            gsql_cmd += 'date_time' + ' '
            gsql_cmd += 'DATETIME'
        else:    
            gsql_cmd += field + ' '
            gsql_cmd += type_translate[type(json_object[field]).__name__]
    
        gsql_cmd += ', '

    return gsql_cmd[:-2] + ')'


def problem_fields_to_str(json_object):

    for key, value in json_object.items():
        if isinstance(value, dict) or value == None:
            json_object[key] = str(value)
        if key == 'date':
            json_object.pop(key)
            json_object['date_time'] = value

    return json_object

def upsert_json(filename, conn, vertex_name, primary_id, n):

    ids = ['']*n
    bodies = ['']*n

    with open(filename, 'r') as f:
        objects = ijson.items(f, '', multiple_values=True, use_float=True)

        i = 0
        count = 0
        for json_object in objects:

            if json_object:
                ids[i]=json_object.pop(primary_id)
                bodies[i]=problem_fields_to_str(json_object)
                count += 1
                i += 1
                if i%n == 0:
                    conn.upsertVertices(vertex_name, list(zip(ids, bodies)))
                    i = 0

            else:
                break
                
    conn.upsertVertices(vertex_name, list(zip(ids[:i], bodies[:i])))

    return count


def create_vertex(shell, json_object, vertex_name, graph_name):

    print(shell.gsql('''
    drop graph {}
    drop vertex {}
    {}
    create graph {} (*)
    ls'''.format(graph_name,
                 vertex_name, 
                 guess_vertex(json_object=json_object, 
                                  vertex_name=vertex_name),
                 graph_name)))

Writing autotigergraph.py


In [11]:
import autotigergraph as atg
import importlib

importlib.reload(atg)

<module 'autotigergraph' from '/home/ubuntu/yelp/autotigergraph.py'>

## Create schema and load data

In [12]:
filename = 'yelp_academic_dataset_business.json'
business, fields = atg.get_first(filename)
business, fields

({'business_id': 'f9NumwFMBDn751xgFiRbNA',
  'name': 'The Range At Lake Norman',
  'address': '10913 Bailey Rd',
  'city': 'Cornelius',
  'state': 'NC',
  'postal_code': '28031',
  'latitude': 35.4627242,
  'longitude': -80.8526119,
  'stars': 3.5,
  'review_count': 36,
  'is_open': 1,
  'attributes': {'BusinessAcceptsCreditCards': 'True',
   'BikeParking': 'True',
   'GoodForKids': 'False',
   'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
   'ByAppointmentOnly': 'False',
   'RestaurantsPriceRange2': '3'},
  'categories': 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping',
  'hours': {'Monday': '10:0-18:0',
   'Tuesday': '11:0-20:0',
   'Wednesday': '10:0-18:0',
   'Thursday': '11:0-20:0',
   'Friday': '11:0-20:0',
   'Saturday': '11:0-20:0',
   'Sunday': '13:0-18:0'}},
 [(0, 'business_id', 'str'),
  (1, 'name', 'str'),
  (2, 'address', 'str'),
  (3, 'city', 'str'),
  (4, 'state', 'str'),
  (5, 'postal_code', 'str'),
  (6,

In [13]:
print(shell.gsql('create graph yelp (*)', options=[]))

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql

Restarting gse gpe restpp ...

Finish restarting services in 12.564 seconds!
The graph yelp is created.



In [14]:
atg.guess_vertex(json_object=business, vertex_name='Business')

'CREATE VERTEX Business (PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING)'

In [15]:
filename = 'yelp_academic_dataset_business.json'
vertex_name = 'Business'
primary_id = 'business_id'

json_object, _ = atg.get_first(filename)
atg.create_vertex(shell=shell, json_object=json_object, vertex_name=vertex_name, graph_name='yelp')
atg.upsert_json(filename=filename, conn=conn, vertex_name=vertex_name, 
                primary_id=primary_id, n=10000)

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The graph yelp is dropped.
The vertex type Business could not be found.
The vertex type Business is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
Edge Types: 

Graphs: 
  - Graph yelp(Business:v)
Jobs: 
Queries: 







209393

In [16]:
filename = 'yelp_academic_dataset_user.json'
vertex_name = 'User'
primary_id = 'user_id'

json_object, _ = atg.get_first(filename)
atg.create_vertex(shell=shell, json_object=json_object, vertex_name=vertex_name, graph_name='yelp')
atg.upsert_json(filename=filename, conn=conn, vertex_name=vertex_name, 
                primary_id=primary_id, n=10000)

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The graph yelp is dropped.
The vertex type User could not be found.
The vertex type User is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment_pho

1968703

In [17]:
filename = 'yelp_academic_dataset_review.json'
vertex_name = 'Review'
primary_id = 'review_id'

json_object, _ = atg.get_first(filename)
atg.create_vertex(shell=shell, json_object=json_object, vertex_name=vertex_name, graph_name='yelp')

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The graph yelp is dropped.
The vertex type Review could not be found.
The vertex type Review is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment

In [18]:
atg.upsert_json(filename=filename, conn=conn, vertex_name=vertex_name, 
                primary_id=primary_id, n=10000)

8021122